# Predicting evasion for a single day : Proof of concept

This adds the static information onto a specefic chosen day

And then imputes the non-static information via looking at historical information

This imputation could more more sopisticated - such as creating a ML algorithm it to predict product validations.  But it will do for now given the time constraint

In [2]:
import pandas as pd

### Importing metric data

In [3]:
Mapping_and_density = pd.read_csv("C:/users/henry chapman/Documents/Coding/Data_science/Project_final/Output/1_Compiling_data/Pipe7/Mapping_and_density.csv")
Stop_visit_score = pd.read_csv("C:/users/henry chapman/Documents/Coding/Data_science/Project_final/Output/1_Compiling_data/Pipe5/Stop_visit_score.csv")
ticket_vals = pd.read_csv("C:/users/henry chapman/Documents/Coding/Data_science/Project_final/Output/1_Compiling_data/Pipe4/Ticket_validations.csv", parse_dates = ["DateTime"])

C:\Users\Henry Chapman\AppData\Local\Temp\ipykernel_26820\118181502.py:3: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  ticket_vals = pd.read_csv("C:/users/henry chapman/Documents/Coding/Data_science/Project_final/Output/1_Compiling_data/Pipe4/Ticket_validations.csv", parse_dates = ["DateTime"])


In [4]:

def Merge_metrics(Day_to_predict, Mapping_and_density, Stop_visit_score):
    # Filter the schedule for the desired day of the week 
    Tartu_schedule = pd.read_csv("c:/users/henry chapman/Documents/Coding/Data_science/Project_final/Output/5_Predicting_future_date/Tartu_schedule.csv")
    # merging date and time into a single date time column
    Tartu_schedule["date"] = pd.to_datetime(Tartu_schedule["date"] + " " + Tartu_schedule["arrival_time"], format = "%Y-%m-%d %H:%M:%S", errors = "coerce")
    Prediction_day = Tartu_schedule[Tartu_schedule["date"].dt.date == Day_to_predict.date()] # filtering for the desired day
    # Adding in pre_2019 column 
    Prediction_day["Pre_2019"] = True

    # Collapsing into the desired granularity - and adding in "Number of buses entering" over that time frame column
    Prediction_day["Binned_day"] = Prediction_day["date"].dt.weekday.map(lambda x : 0 if x < 5 else 1)
    Prediction_day["Binned_time"] = Prediction_day["date"].dt.round("120min")
    # Counting the number of buses which enter that bus stop in that time window
    Collapsed = Prediction_day.groupby(["Stop_ID", "Binned_day", "Binned_time", "Pre_2019"]).size().reset_index(name = "Number_entering_buses")


    # Merging with metrics 
    Merge_1 = Collapsed.merge(Mapping_and_density, left_on = ["Stop_ID", "Pre_2019"], right_on = ["Stop_ID", "Pre_2019"])
    Merge_2 = Merge_1.merge(Stop_visit_score, left_on = ["Stop_ID", "Pre_2019"], right_on = ["Stop_ID", "Pre_2019"])

    return Merge_2


Day_to_predict = pd.Timestamp("2025.12.11")
Merged_metrics = Merge_metrics(Day_to_predict, Mapping_and_density, Stop_visit_score)

Merged_metrics.head()
    

C:\Users\Henry Chapman\AppData\Local\Temp\ipykernel_26820\2261354665.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Prediction_day["Pre_2019"] = True
C:\Users\Henry Chapman\AppData\Local\Temp\ipykernel_26820\2261354665.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Prediction_day["Binned_day"] = Prediction_day["date"].dt.weekday.map(lambda x : 0 if x < 5 else 1)
C:\Users\Henry Chapman\AppData\Local\Temp\ipykernel_26820\2261354665.py:12: SettingWithCopyWarning: 
A value is trying to be set on a co

,Stop_ID,Binned_day,Binned_time,Pre_2019,Number_entering_buses,density,NIMI,Visiting_score,Rel_Visiting_score
0,7800431-1,0,2025-12-11 16:00:00,True,1,0,Ränilinna,1,0.03125
1,7800528-1,0,2025-12-11 06:00:00,True,4,0,Ihaste,1,0.03125
2,7800528-1,0,2025-12-11 08:00:00,True,4,0,Ihaste,1,0.03125
3,7800528-1,0,2025-12-11 10:00:00,True,4,0,Ihaste,1,0.03125
4,7800528-1,0,2025-12-11 12:00:00,True,4,0,Ihaste,1,0.03125


In [5]:
def Impute_validation_count(Merge_metrics, ticket_vals, Day_to_predict):
    MM_copy = Merged_metrics.copy()
    # add all neccasary columns 
    MM_copy["Day_of_week"] = MM_copy["Binned_time"].dt.weekday
    MM_copy["Time"] = MM_copy["Binned_time"].dt.time

    TV_vals = ticket_vals.copy()
    TV_vals = TV_vals[TV_vals["Pre_2019"] == False]

    # Filter TV_vals for all of the previous days and month, across the other years 
    TV_vals_filtered = TV_vals[
        (TV_vals["DateTime"].dt.day == Day_to_predict.day) &
        (TV_vals["DateTime"].dt.month == Day_to_predict.month)
        ]
    
    # Adding in weekend split column 
    TV_vals_filtered["Binned_day"] = TV_vals_filtered["DateTime"].dt.weekday.map(lambda x : 0 if x < 5 else 1)
    # Adding in Binned time coolumns
    TV_vals_filtered["Binned_time"] = TV_vals_filtered["DateTime"].dt.round("120min")
    TV_vals_filtered["Binned_time"] = TV_vals_filtered["Binned_time"].dt.time

    print(TV_vals_filtered.head().to_string())
    print(MM_copy.head().to_string())

    Mean_vals = TV_vals_filtered.groupby(["Stop_ID", "Binned_day", "Binned_time"])["Validation_Count"].mean().reset_index(name = "Validation_Count_mean")

    Merge = MM_copy.merge(Mean_vals, left_on = ["Stop_ID", "Binned_day", "Time"], right_on = ["Stop_ID", "Binned_day", "Binned_time"])

    Final = Merge[["Stop_ID", "Binned_day", "Binned_time_x", "Pre_2019", "density", "NIMI", "Visiting_score", "Rel_Visiting_score", "Validation_Count_mean", "Number_entering_buses"]]
    return Final
        
    
Final = Impute_validation_count(Merge_metrics, ticket_vals, Day_to_predict)


                    DateTime Line_clean    Stop_ID  Pre_2019  Validation_Count  Binned_day Binned_time
13586589 2019-12-11 00:06:00          1  7820277-1     False               1.0           0    00:00:00
13586590 2019-12-11 00:09:00          1  7820243-1     False               2.0           0    00:00:00
13586591 2019-12-11 00:10:00          1  7820061-1     False               1.0           0    00:00:00
13586592 2019-12-11 00:14:00          1  7820187-1     False               6.0           0    00:00:00
13586593 2019-12-11 00:12:00          1  7820178-1     False              12.0           0    00:00:00
     Stop_ID  Binned_day         Binned_time  Pre_2019  Number_entering_buses  density       NIMI  Visiting_score  Rel_Visiting_score  Day_of_week      Time
0  7800431-1           0 2025-12-11 16:00:00      True                      1        0  Ränilinna               1             0.03125            3  16:00:00
1  7800528-1           0 2025-12-11 06:00:00      True              

C:\Users\Henry Chapman\AppData\Local\Temp\ipykernel_26820\3365431974.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TV_vals_filtered["Binned_day"] = TV_vals_filtered["DateTime"].dt.weekday.map(lambda x : 0 if x < 5 else 1)
C:\Users\Henry Chapman\AppData\Local\Temp\ipykernel_26820\3365431974.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TV_vals_filtered["Binned_time"] = TV_vals_filtered["DateTime"].dt.round("120min")
C:\Users\Henry Chapman\AppData\Local\Temp\ipykernel_26820\3365431974.py:20: Set

In [6]:
import numpy as np
def Encoding_variables(Final):
    Final["Time_min"] = (Final["Binned_time_x"].dt.hour)*60
    
    Final["time_sin"] = np.sin(2*np.pi*Final["Time_min"]/1440)
    Final["time_cos"] = np.cos(2*np.pi*Final["Time_min"]/1440)
    
    Final.drop(columns = ["Time_min"], inplace = True)


    # one hot encoding day, inlcuding day of the week (if it exists)    

    Final = pd.get_dummies(Final, columns = ["NIMI"])
    # Addining in binned day manually - WARNING THIS ASSUMES THAT WE ARE PREDICTING A WEEK DAY
    Final["Binned_day_0"] = True
    Final["Binned_day_1"] = False

        

    Final.drop(columns = ["Binned_day"], inplace = True)
    return Final

In [7]:
Encoded = Encoding_variables(Final)
Encoded.head()

,Stop_ID,Binned_time_x,Pre_2019,density,Visiting_score,Rel_Visiting_score,Validation_Count_mean,Number_entering_buses,time_sin,time_cos,...,NIMI_Ränilinna,NIMI_Supilinna,NIMI_Tammelinna,NIMI_Tähtvere,NIMI_Vaksali,NIMI_Variku,NIMI_Veeriku,NIMI_Ülejõe,Binned_day_0,Binned_day_1
0,7800528-1,2025-12-11 08:00:00,True,0,1,0.03125,1.833333,4,8.660254e-01,-0.500000,...,False,False,False,False,False,False,False,False,True,False
1,7800528-1,2025-12-11 10:00:00,True,0,1,0.03125,1.400000,4,5.000000e-01,-0.866025,...,False,False,False,False,False,False,False,False,True,False
2,7800528-1,2025-12-11 12:00:00,True,0,1,0.03125,1.000000,4,1.224647e-16,-1.000000,...,False,False,False,False,False,False,False,False,True,False
3,7800528-1,2025-12-11 14:00:00,True,0,1,0.03125,1.000000,4,-5.000000e-01,-0.866025,...,False,False,False,False,False,False,False,False,True,False
4,7800528-1,2025-12-11 16:00:00,True,0,1,0.03125,1.750000,4,-8.660254e-01,-0.500000,...,False,False,False,False,False,False,False,False,True,False


# Exporting

In [8]:
Encoded.to_csv("C:/users/henry chapman/Documents/Coding/Data_science/Project_final/Output/5_Predicting_future_date/Pipe2/Pred_day.csv", index = False)